In [ ]:
from IPython.display import HTML
from functools import partial
from libs.utils import run_test
from libs.sollutions.ans1 import solved1

import torch
import einops
import matplotlib.pyplot as plt
import plotly
import plotly.graph_objects as go

import plotly.express as px

import ipywidgets as widgets
from IPython.display import display

from ipywidgets import interact

%matplotlib nbagg


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.2.6 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "/home/adi/anaconda3/lib/python3.12/runpy.py", line 198, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/home/adi/anaconda3/lib/python3.12/runpy.py", line 88, in _run_code
    exec(code, run_globals)
  File "/home/adi/anaconda3/lib/python3.12/site-packages/ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "/home/adi/anaconda3/lib/python3.12/site-packages/traitlets/config/application.py", line 1075, in launch_instance
    app.start()
  File "/home/adi/ana

ImportError: 
A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.2.6 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.




A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.2.6 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "/home/adi/anaconda3/lib/python3.12/runpy.py", line 198, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/home/adi/anaconda3/lib/python3.12/runpy.py", line 88, in _run_code
    exec(code, run_globals)
  File "/home/adi/anaconda3/lib/python3.12/site-packages/ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "/home/adi/anaconda3/lib/python3.12/site-packages/traitlets/config/application.py", line 1075, in launch_instance
    app.start()
  File "/home/adi/ana

ImportError: 
A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.2.6 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.



In [ ]:
%load_ext autoreload
%autoreload 1

%aimport libs.sollutions
%aimport libs.utils

In [ ]:
%aimport

In [ ]:
## Quiz 1

In [ ]:
def quiz1():
    return "False"

In [ ]:
run_test(quiz1,solved1)

In [ ]:
import os
import random
available_images = []
for root, dirs, files in os.walk("./libs/datasets_and_models/sample_animals10/"):
    for fname in files:
        available_images.append(os.path.join(root, fname))
random.shuffle(available_images)

In [ ]:
img_path_dropdown = widgets.Dropdown(
    options=available_images,
    value=available_images[0],
    description='choose_image',
    disabled=False,
)
display(img_path_dropdown)


# img_path = "./libs/datasets_and_models/sample_animals10/dog/OIP-_GmIl90tmXb9c72orTGKEQHaJ4.jpeg"
from torchvision.io import decode_image

In [ ]:
img = decode_image(img_path_dropdown.value)
img = einops.rearrange(img,"c h w -> 1 c h w")
img = torch.nn.functional.interpolate(img,size=(256,256))
# plt.imshow(einops.rearrange(img,"1 c h w -> h w c"))
fig1 = px.imshow(einops.rearrange(img,"1 c h w -> h w c"))
fig1.show()

img = torch.nn.functional.interpolate(img,size=(128,128))


In [ ]:
from libs.saliency_methods.load_model import get_conv_model, test_model

model = get_conv_model()
model.eval()

# test_model(model)

In [ ]:
global activations
global gradients

target_layer = model.conv3
logit_idx = 0

def save_activation(module,input,output):
    global activations
    activations = output.detach()

def save_grad(module,grad_input,grad_output):
    global gradients
    gradients = grad_output[0].detach()


target_layer.register_forward_hook(save_activation)
target_layer.register_backward_hook(save_grad)

out = model(img.to(torch.float32))
desired_logit = out[0][0]
desired_logit.backward(retain_graph=True)

print(out,torch.argmax(out))

# print(model,target_layer)

In [ ]:
activations.shape

In [ ]:
gradients.shape

In [ ]:
slider = widgets.IntSlider(0,0,activations.shape[0]-1,step=1,description="Output Mode")

In [ ]:
upscaled_gradients = torch.nn.functional.interpolate(gradients,(256,256))
activations = torch.nn.functional.interpolate(activations,(256,256))
img = torch.nn.functional.interpolate(img,(256,256))

upscaled_gradients = einops.rearrange(upscaled_gradients,"1 o h w -> o h w")
activations = einops.rearrange(activations,"1 o h w -> o h w")
img = einops.rearrange(img,"1 c h w -> c h w")

In [ ]:

def plot_grads(sal,mode_idx):
    map_idx = sal[mode_idx,:,:]
    fig = px.imshow(einops.rearrange(img,"c h w -> h w c"))

    fig.add_trace(
        go.Heatmap(
            z=map_idx,
            colorscale='plasma', # Use a distinct color scale
            opacity=0.6,         # Key: Set transparency (adjust this value!)
            showscale=True,
            name='Saliency'
        )
    )
    
    fig.show()



widgets.interactive(
    lambda mode_idx: plot_grads(upscaled_gradients, mode_idx),
    mode_idx=slider
)


In [ ]:
print(activations.shape, upscaled_gradients.shape)
weighted_activations = activations * upscaled_gradients

weighted_activations = torch.relu(weighted_activations)
weighted_activations = torch.sum(weighted_activations,dim=0,keepdims=True)

In [ ]:
slider2 = widgets.IntSlider(0,0,activations.shape[0]-1,step=1,description="Output Mode")
widgets.interactive(
    lambda mode_idx: plot_grads(weighted_activations, mode_idx),
    mode_idx=slider2
)